In [94]:
import os 
import pandas as pd
from openai import OpenAI, api_key
from dotenv import load_dotenv
from pathlib import Path
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DataFrameLoader
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings




In [91]:
load_dotenv()

True

In [80]:
init_prompt =  open('data/init_prompt.txt').read()

# RAG建構 

In [81]:

# 加载 Excel 文件
file_path = "data/回饋規則.xlsx"
data = pd.read_excel(file_path)
# 查看数据
print(data.head())


             球路類型  結果                        原因                      改善建議
0         Pull左飛球  失誤          上桿(P2~3)時，角度過於陡峭          上桿(P2~3)時，肩膀往右轉動
1         Pull左飛球  失誤                    桿頭頂點過高                    肩膀往右轉動
2         Pull左飛球  失誤  下桿角度過於陡峭，左手腕過度外展，肩關節伸展抬起  下桿時，左手臂打直、左手腕維持固定、肩膀自然旋轉
3         Pull左飛球  失誤            桿面關閉，擊球點位於球的外側       擊球時，左手腕維持固定，注意擊球點位置
4  Pull Hook左拉左曲球  失誤          上桿(P2~3)時，角度過於陡峭          上桿(P2~3)時，肩膀往右轉動


In [82]:
documents = [
    Document(
        page_content=" | ".join(map(str, row.values)), 
        metadata={"row_index": index}
    )
    for index, row in data.iterrows()
]

print(f"生成了 {len(documents)} 個文檔")
print(documents[0])

生成了 28 個文檔
page_content='Pull左飛球 | 失誤 | 上桿(P2~3)時，角度過於陡峭 | 上桿(P2~3)時，肩膀往右轉動' metadata={'row_index': 0}


In [97]:
# 使用 OpenAI Embeddings 对文档进行向量化
openai_api_key = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# 创建向量数据库
vectorstore = Chroma.from_documents(documents, embeddings)





ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [98]:
#嘗試只使用excel轉文字

In [112]:
from openai import OpenAI
import pandas as pd


# 讀取 CSV 檔案
csv_file_path = "data/回饋規則.xlsx"
data = pd.read_excel(csv_file_path)

# 將數據轉換為文字格式（或提取特定欄位）
text_representation = data.head(5).to_string()  # 取前5行並轉換為文字

# 定義提示字串
prompt = f"""
{init_prompt}
以下是一些來自回饋規則檔案的數據：
{text_representation}

"""

# 呼叫 OpenAI API
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": prompt},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "球速:66.43、發射角度:7.198、發射方向:-6.172、飛行距離:179、ClubAngleFace:-15、ClubAnglePath:6.901"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://cdn.discordapp.com/attachments/727209373612638362/1315702149514137763/combined_186382.jpg?ex=67585e9d&is=67570d1d&hm=fcb27e71292ea87bf197c471494e078cdd2e61341dee5ddb67c9ca2d7ccab49b&",
                    }
                },
            ],
        }
    ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content='```json\n{\n  "球路": "Pull Left Shot",\n  "原因": "上桿(P2~3)時，肩膀角度過於陡峭，使得桿面關閉。",\n  "建議": "在我們全面的揮杆分析中，您的揮杆動作與教練最相似的地方在於擊球階段(T)，請繼續保持。但在上桿(P2~3)時，建議您調整肩膀的位置，讓肩膀稍微向右轉動，這會有助於改善桿面的開放角度，從而提高擊球精準度。"\n}\n```', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
